In [4]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [3]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/lme4_1.1-27.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 3311365 bytes (3.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

In [5]:
# Load data
conversations = pd.read_csv("results/contingency/conversations.csv")
conversations_melted = pd.read_csv("results/contingency/conversations_melted.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)
conversations_melted.replace({False: 0, True: 1}, inplace=True)

conversations.head()


,utterance_id,speaker_code,transcript_raw,tokens,pos,start_time,end_time,age,corpus,transcript_file,...,follow_up_transcript_raw,follow_up_start_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response,pos_feedback,response_is_clarification_request
0,9,CHI,&=vocalize .,['.'],['none'],15320.0,16139.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,&=vocalize .,19069.0,1,0,YY,-252.0,2930.0,1,1,0
1,50,CHI,&=babble .,['.'],['none'],82170.0,82989.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,Dada .,86388.0,1,0,YY,712.0,3399.0,1,1,0
2,52,CHI,Dada .,"['dada', '.']",['n:prop'],86388.0,87007.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,&=babble .,89517.0,1,0,YY,572.0,2510.0,1,1,0
3,157,CHI,&=vocalize .,['.'],['none'],367633.0,368084.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,&=vocalize .,370041.0,1,0,YY,528.0,1957.0,1,1,0
4,160,CHI,&=vocalize .,['.'],['none'],372007.0,372211.0,12,Brent,/home/mitja/data/CHILDES/Brent/c1/000917.cha,...,&=vocalize .,381680.0,1,0,YY,6830.0,9469.0,0,0,0


In [6]:
# Some children have very few data points:
counts = conversations.groupby("child_name").size()
counts

child_name
Brent_Alexander                 214
Brent_Allen                       3
Brent_Brooklyn                   11
Brent_Dillon                    663
Brent_Henry                     208
Brent_Jacob_Abernathy             4
Brent_Jaylen                     36
Brent_Maggie                    157
Brent_Miranda                   197
Brent_Morgan                    525
Brent_Tabitha                   215
Brent_Tabitha_Sims                3
Brent_Timothy                   371
Brent_Tyrese                      6
Brent_Vas                        12
Brent_Vas_Coleman                93
Brent_Xavier                     11
MPI-EVA-Manchester_Eleanor    37541
MPI-EVA-Manchester_Fraser     91592
Thomas_Brian                    280
Thomas_Thomas                 94152
dtype: int64

In [7]:
# Exclude children with less than 100 datapoints:
# child_names_enough_data = [name for name, count in counts.items() if count > 100]
# print(len(conversations))
# conversations = conversations[conversations.child_name.isin(child_names_enough_data)]
# print(len(conversations))

In [8]:
# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [14]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ is_intelligible * age_normalized + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
173345.9 173397.7 -86668.0 173335.9   231536 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4872  0.3109  0.3410  0.3895  0.9238 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.2714   0.5209  
Number of obs: 231541, groups:  child_name, 22

Fixed effects:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     0.78918    0.10351   7.624 2.46e-14 ***
is_intelligible                 0.79457    0.04085  19.449  < 2e-16 ***
age_normalized                  0.66146    0.06743   9.810  < 2e-16 ***
is_intelligible:age_normalized  0.44585    0.07488   5.954 2.62e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.

### Clarification requests

In [17]:
%%R -i conversations
library(lme4)

# TODO: filter out no response cases?
m_caregiver_contingency<-glmer('response_is_clarification_request ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
response_is_clarification_request ~ is_intelligible * age_normalized +  
    (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
 15518.4  15570.2  -7754.2  15508.4   231536 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-0.283 -0.075 -0.072 -0.031 36.393 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 1.056    1.028   
Number of obs: 231541, groups:  child_name, 22

Fixed effects:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     -3.7426     0.2366 -15.819  < 2e-16 ***
is_intelligible                 -2.6123     0.1223 -21.367  < 2e-16 ***
age_normalized                  -2.0319     0.2035  -9.987  < 2e-16 ***
is_intelligible:age_normalized   1.6127     0.2326   6.934 4.09e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.

### Combined (Positive Feedback = No pause, no clarification request)

In [18]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('pos_feedback ~ is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: pos_feedback ~ is_intelligible * age_normalized + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
177320.8 177372.6 -88655.4 177310.8   231536 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2027  0.3113  0.3407  0.4003  0.9944 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.3022   0.5497  
Number of obs: 231541, groups:  child_name, 22

Fixed effects:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     0.62859    0.09246   6.798 1.06e-11 ***
is_intelligible                 1.00836    0.04093  24.636  < 2e-16 ***
age_normalized                  0.89248    0.06463  13.809  < 2e-16 ***
is_intelligible:age_normalized  0.19024    0.07467   2.548   0.0108 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.

## Effect of communicative feedback
### Positive Feedback: Timing

In [9]:
%%R -i conversations
library(lme4)

# conversations_child_intelligible = subset(conversations, is_intelligible==1)
# TODO: only intelligible ones?

m_child_contingency<-glmer('follow_up_is_intelligible ~ has_response * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_child_contingency))

R[write to console]: Loading required package: Matrix



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_intelligible ~ has_response * age + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
116475.9 116527.6 -58233.0 116465.9   226289 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.5032  0.2208  0.2588  0.2853  9.1575 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 9.025    3.004   
Number of obs: 226294, groups:  child_name, 21

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)       0.05016    0.11687   0.429    0.668    
has_response      1.28343    0.04121  31.144   <2e-16 ***
age               2.17321    0.06646  32.700   <2e-16 ***
has_response:age -1.00427    0.07487 -13.414   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) hs_rsp age   

### Negative Feedback: Clarification requests

In [12]:
%%R -i conversations_melted
library(lme4)

conversations_melted_cr = subset(conversations_melted, response_is_clarification_request == 1)


m_child_contingency<-glmer('is_intelligible ~ is_follow_up * age + (1 | child_name) + (1 | is_follow_up)', data=conversations_melted_cr, family=binomial)
print(summary(m_child_contingency))

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_intelligible ~ is_follow_up * age + (1 | child_name) + (1 |  
    is_follow_up)
   Data: conversations_melted_cr

     AIC      BIC   logLik deviance df.resid 
  2516.1   2550.4  -1252.0   2504.1     2262 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1473 -0.8499  0.4280  0.6868  1.8475 

Random effects:
 Groups       Name        Variance Std.Dev.
 child_name   (Intercept) 5.069    2.251   
 is_follow_up (Intercept) 0.000    0.000   
Number of obs: 2268, groups:  child_name, 12; is_follow_up, 2

Fixed effects:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -4.2389374  1.1742263  -3.610 0.000306 ***
is_follow_up      1.0963349  0.5262573   2.083 0.037227 *  
age               0.0796228  0.0116576   6.830 8.48e-12 ***
is_follow_up:age -0.0008143  0.0181168  -0.045 0.964152    
---
Signif. codes:  0 ‘***’ 0.

In [29]:
%%R -i data_new
library(lme4)

m_child_contingency<-glmer('is_intelligible ~ response_is_clarification_request * trial + (1 | child_name) + (1 | trial)', data=data_new, family=binomial)
print(summary(m_child_contingency))

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_intelligible ~ response_is_clarification_request * trial +  
    (1 | child_name) + (1 | trial)
   Data: data_new

      AIC       BIC    logLik  deviance  df.resid 
 337864.3  337930.8 -168926.1  337852.3    479094 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.1152  0.3116  0.3541  0.3612  7.2539 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 8.11     2.848   
 trial      (Intercept) 0.00     0.000   
Number of obs: 479100, groups:  child_name, 21; trial, 2

Fixed effects:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                              0.81832    0.04948  16.539  < 2e-16
response_is_clarification_request       -1.81910    0.04132 -44.024  < 2e-16
trial                                   -0.03997    0.00895  -4.466 7.97e-06
response_is_clarificat